In [1]:
# pip install pyspark

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Practice').master("local[*]").getOrCreate()

24/08/14 11:16:55 WARN Utils: Your hostname, PC-390 resolves to a loopback address: 127.0.1.1; using 192.168.15.114 instead (on interface enp4s0)
24/08/14 11:16:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/14 11:16:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
customer = spark.read.csv("Ecommerce Database/Customers.csv",
                          inferSchema=True, header=True)

In [6]:
customer.show()

+----------+-----------+---------+-------------------+-----------+----------------+-------------+----------+----------+--------------------+-------------------+
|CustomerID|  FirstName| LastName|      Date_of_Birth|       City|           State|      Country|PostalCode|     Phone|               Email|        DateEntered|
+----------+-----------+---------+-------------------+-----------+----------------+-------------+----------+----------+--------------------+-------------------+
|     57081|      James|    Smith|1987-03-26 00:00:00|   New York|        New York|United States|    280862|9638483934|James.Smith@gmail...|2020-01-02 00:00:00|
|     57082|     Robert|Downey Jr|1973-05-24 00:00:00|   New York|        New York|United States|    376573|6588282115|Robert.Downey Jr@...|2020-01-06 00:00:00|
|     57083|       John| Williams|1990-04-14 00:00:00|    Chicago|        Illinois|United States|    485629|7641021429|John.Williams@gma...|2020-01-11 00:00:00|
|     57084|    Michael|  Johnson|

In [7]:
customer.count()

525

In [8]:
len(customer.columns)

11

In [9]:
customer.select("FirstName", "LastName", "Country").show(10)

+---------+---------+-------------+
|FirstName| LastName|      Country|
+---------+---------+-------------+
|    James|    Smith|United States|
|   Robert|Downey Jr|United States|
|     John| Williams|United States|
|  Michael|  Johnson|    Australia|
|    Steve| Williams|      Germany|
|    David|  Beckham|      Austria|
|  Richard|    Brown|United States|
|   Joseph|    James|  Netherlands|
|   Thomas|    Jones|United States|
|  Charles|     King|       Poland|
+---------+---------+-------------+
only showing top 10 rows



In [10]:
customer.groupBy("Country").count().show()

+----------------+-----+
|         Country|count|
+----------------+-----+
|          Russia|   24|
|         Germany|   29|
|          France|   35|
|          Greece|   28|
|         Belgium|   27|
|   United States|   66|
|           India|   40|
|           Italy|   14|
|         Ireland|   27|
|     Switzerland|   33|
|     New Zealand|    5|
|          Poland|   19|
|        Portugal|   37|
|       Australia|   30|
|         Romania|   20|
|         Austria|   32|
|    South Africa|    5|
|Northern Ireland|   30|
|     Netherlands|   24|
+----------------+-----+



In [11]:
customer.groupBy("Country").count().orderBy("count", ascending=False).show()

+----------------+-----+
|         Country|count|
+----------------+-----+
|   United States|   66|
|           India|   40|
|        Portugal|   37|
|          France|   35|
|     Switzerland|   33|
|         Austria|   32|
|       Australia|   30|
|Northern Ireland|   30|
|         Germany|   29|
|          Greece|   28|
|         Belgium|   27|
|         Ireland|   27|
|          Russia|   24|
|     Netherlands|   24|
|         Romania|   20|
|          Poland|   19|
|           Italy|   14|
|     New Zealand|    5|
|    South Africa|    5|
+----------------+-----+



In [12]:
from pyspark.sql import functions as F
customer.groupBy(F.year("DateEntered").alias("Year"),
                 F.month("DateEntered").alias("Month")).count().alias("Number of customers").orderBy("Year", "Month").show()

+----+-----+-----+
|Year|Month|count|
+----+-----+-----+
|2020|    1|   10|
|2020|    2|   23|
|2020|    3|   30|
|2020|    4|   30|
|2020|    5|   21|
|2020|    6|   20|
|2020|    7|   10|
|2020|    8|   15|
|2020|    9|   22|
|2020|   10|   18|
|2020|   11|   10|
|2020|   12|   10|
|2021|    1|   40|
|2021|    2|   41|
|2021|    3|   39|
|2021|    4|   25|
|2021|    5|   23|
|2021|    6|   23|
|2021|    7|   20|
|2021|    8|   22|
+----+-----+-----+
only showing top 20 rows



In [13]:
customer.createTempView("customer")

In [14]:
spark.sql('SELECT * FROM CUSTOMER').show()

+----------+-----------+---------+-------------------+-----------+----------------+-------------+----------+----------+--------------------+-------------------+
|CustomerID|  FirstName| LastName|      Date_of_Birth|       City|           State|      Country|PostalCode|     Phone|               Email|        DateEntered|
+----------+-----------+---------+-------------------+-----------+----------------+-------------+----------+----------+--------------------+-------------------+
|     57081|      James|    Smith|1987-03-26 00:00:00|   New York|        New York|United States|    280862|9638483934|James.Smith@gmail...|2020-01-02 00:00:00|
|     57082|     Robert|Downey Jr|1973-05-24 00:00:00|   New York|        New York|United States|    376573|6588282115|Robert.Downey Jr@...|2020-01-06 00:00:00|
|     57083|       John| Williams|1990-04-14 00:00:00|    Chicago|        Illinois|United States|    485629|7641021429|John.Williams@gma...|2020-01-11 00:00:00|
|     57084|    Michael|  Johnson|

In [15]:
spark.sql("""
            SELECT
              YEAR('DateEntered') AS Year,
              Month('DateEntered') AS Month,
              COUNT(*) AS Number_of_customers
              FROM customer
              GROUP BY Year, Month
              ORDER BY Year, Month
              """).show()

+----+-----+-------------------+
|Year|Month|Number_of_customers|
+----+-----+-------------------+
|NULL| NULL|                525|
+----+-----+-------------------+



In [16]:
Order = spark.read.csv("Ecommerce Database/Orders.csv", inferSchema=True, header=True)

In [17]:
Order.show()

+-------+----------+---------+----------+---------+----------+------------+------------------+
|OrderID|CustomerID|PaymentID| OrderDate|ShipperID|  ShipDate|DeliveryDate|Total_order_amount|
+-------+----------+---------+----------+---------+----------+------------+------------------+
|7655500|     57083|        2|12-01-2020|        7|13-01-2020|  19-01-2020|           25112.0|
|7655501|     57086|        3|20-01-2020|        2|24-01-2020|  27-01-2020|           22453.0|
|7655502|     57086|        4|06-02-2020|        7|11-02-2020|  21-02-2020|           13293.0|
|7655503|     57088|        4|09-02-2020|        1|13-02-2020|  26-02-2020|           16063.0|
|7655504|     57090|        4|11-02-2020|        3|15-02-2020|  20-02-2020|           15193.0|
|7655505|     57094|        4|15-02-2020|        1|24-02-2020|  27-02-2020|           13581.0|
|7655506|     57092|        4|20-02-2020|        8|23-02-2020|  27-02-2020|           74120.0|
|7655507|     57095|        6|23-02-2020|        8

In [18]:
customerOrder = customer.join(Order, on="CustomerID", how="inner").show()

+----------+---------+---------+-------------------+---------+----------------+----------------+----------+----------+--------------------+-------------------+-------+---------+----------+---------+----------+------------+------------------+
|CustomerID|FirstName| LastName|      Date_of_Birth|     City|           State|         Country|PostalCode|     Phone|               Email|        DateEntered|OrderID|PaymentID| OrderDate|ShipperID|  ShipDate|DeliveryDate|Total_order_amount|
+----------+---------+---------+-------------------+---------+----------------+----------------+----------+----------+--------------------+-------------------+-------+---------+----------+---------+----------+------------+------------------+
|     57083|     John| Williams|1990-04-14 00:00:00|  Chicago|        Illinois|   United States|    485629|7641021429|John.Williams@gma...|2020-01-11 00:00:00|7655500|        2|12-01-2020|        7|13-01-2020|  19-01-2020|           25112.0|
|     57086|    David|  Beckham|

In [19]:
import pyspark.sql.functions as F

customerOrder = customer.join(Order, on="CustomerID", how="inner")

# Use the correct column name 'Total_order_amount'
customerOrder.groupBy("Country").agg(
    F.round(F.sum("Total_order_amount"),
            0).alias("Revenue")
).orderBy("Revenue", ascending=False).show()

+----------------+-----------+
|         Country|    Revenue|
+----------------+-----------+
|   United States|1.1787059E7|
|           India|  7813408.0|
|        Portugal|  6626170.0|
|       Australia|  5987945.0|
|         Austria|  5629163.0|
|     Switzerland|  5417082.0|
|          France|  5364255.0|
|Northern Ireland|  5129955.0|
|          Greece|  4994220.0|
|     Netherlands|  4742809.0|
|         Ireland|  4645891.0|
|         Germany|  4541393.0|
|          Russia|  4404334.0|
|         Belgium|  4069739.0|
|         Romania|  3891355.0|
|          Poland|  3050020.0|
|           Italy|  1915701.0|
|     New Zealand|   956542.0|
|    South Africa|   952107.0|
+----------------+-----------+

